# VGGNet


## 실습 목표
-----
- VGGNet을 사용하여 이미지를 학습하고 10개의 카테고리를 갖는 이미지를 분류하는 이미지 분류기를 생성 (데이터셋: [CIFAR](https://pytorch.org/vision/0.9/datasets.html#cifar))
- Pre-training 모델의 사용방법을 이해

## CIFAR Classifier(VGGNet)
----
CIFAR 데이터셋을 사용하여 이미지에 포함된 object가 무엇인지 분류하는 이미지 분류기를 생성

### [Step1] Load libraries & Datasets

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch import nn

from torchvision import datasets
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_pil_image

### [Step2] Data preprocessing

불러온 이미지의 증강을 통해 학습 정확도를 향상시킴

* RandomCrop
* RandomHorizontalFlip
* Normalize

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_img = datasets.CIFAR10(
    root = 'data',
    train = True,
    download = True,
    transform = transform,
)

test_img = datasets.CIFAR10(
    root = 'data',
    train = False,
    download = True,
    transform = transform,
)

100%|██████████| 170M/170M [00:03<00:00, 43.4MB/s]


### [Step3] Set hyperparameters

In [3]:
EPOCH = 10
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", DEVICE)

Using Device: cuda


### [Step4] Create DataLoader

In [4]:
train_loader = DataLoader(train_img, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_img, batch_size = BATCH_SIZE, shuffle = False)

### [Step5] Set Network Structure

In [5]:
# # Model
# cfg = {
#     'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
#     'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
#     'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
#     'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
# }

# class VGG(nn.Module):
#     def __init__(self, vgg_name):
#         super(VGG, self).__init__()
#         self.features = self._make_layers(cfg[vgg_name])
#         self.classifier = nn.Sequential(
#             nn.Linear(512 * 1 * 1, 360),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(360, 100),
#             nn.ReLU(inplace=True),
#             nn.Dropout(),
#             nn.Linear(100, 10),
#         )

#     def forward(self, x):
#         out = self.features(x)
#         out = out.view(out.size(0), -1)
#         out = self.classifier(out)
#         return out

#     def _make_layers(self, cfg):
#         layers = []
#         in_channels = 3
#         for x in cfg:
#             if x == 'M':
#                 layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
#             else:
#                 layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
#                            nn.BatchNorm2d(x),
#                            nn.ReLU(inplace=True)]
#                 in_channels = x
#         return nn.Sequential(*layers)

In [6]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])

        dummy_input = torch.zeros(1, 3, 224, 224)  # Input image size
        self.flattened_size = self._get_conv_output_size(dummy_input)

        self.classifier = nn.Sequential(
            nn.Linear(self.flattened_size, 360),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(360, 100),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(100, 10),
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)  # Flatten the output
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        return nn.Sequential(*layers)

    def _get_conv_output_size(self, x):
        with torch.no_grad():
            x = self.features(x)
            return int(torch.prod(torch.tensor(x.size()[1:])))  # Flatten the output size

### [Step6] Create Model instance

In [7]:
model = VGG('VGG16').to(DEVICE)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

### [Step7] Model compile

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = LEARNING_RATE, momentum=0.9)

### [Step8] Set train loop

In [9]:
def train(train_loader, model, loss_fn, optimizer):
    model.train()

    size = len(train_loader.dataset)

    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        pred = model(X)

        # 손실 계산
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')

### [Step9] Set test loop

In [10]:
def test(test_loader, model, loss_fn):
    model.eval()

    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

### [Step10] Run model

In [12]:
for i in range(EPOCH) :
    print(f"Epoch {i+1} \n------------------------")
    train(train_loader, model, loss, optimizer)
    test(test_loader, model, loss)
print("Done!")

Epoch 1 
------------------------
loss: 0.810061  [    0]/50000
loss: 0.944061  [ 3200]/50000
loss: 0.956602  [ 6400]/50000
loss: 1.058247  [ 9600]/50000
loss: 1.131839  [12800]/50000
loss: 1.069352  [16000]/50000
loss: 0.699828  [19200]/50000
loss: 0.887069  [22400]/50000
loss: 1.125874  [25600]/50000
loss: 0.673224  [28800]/50000
loss: 0.903553  [32000]/50000
loss: 0.770246  [35200]/50000
loss: 0.770497  [38400]/50000
loss: 1.183412  [41600]/50000
loss: 0.652114  [44800]/50000
loss: 0.858100  [48000]/50000
Test Error: 
 Accuracy: 73.8%, Avg loss: 0.778046

Epoch 2 
------------------------
loss: 0.831051  [    0]/50000
loss: 1.114458  [ 3200]/50000
loss: 0.621431  [ 6400]/50000
loss: 1.006267  [ 9600]/50000
loss: 0.542757  [12800]/50000
loss: 0.884232  [16000]/50000
loss: 0.554615  [19200]/50000
loss: 0.674888  [22400]/50000
loss: 0.445910  [25600]/50000
loss: 0.443742  [28800]/50000
loss: 0.789550  [32000]/50000
loss: 0.497531  [35200]/50000
loss: 0.823865  [38400]/50000
loss: 0.909

## CIFAR Classifier(Pretrained VGGNet)
----
ImageNet 데이터로 학습한 VGGNet을 사용하여 주어진 데이터 셋에서 사용할 수 있도록 Fine tuning


In [13]:
from torchvision import models

vgg16 = models.vgg16(pretrained=True)
vgg16.to(DEVICE)
print(vgg16)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 65.4MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
vgg16.classifier[6].out_features = 10

for param in vgg16.features.parameters():
    param.requires_grad = False

In [15]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg16.classifier.parameters(), lr = LEARNING_RATE, momentum=0.9)

In [ ]:
for i in range(EPOCH) :
    print(f"Epoch {i+1} \n------------------------")
    train(train_loader, vgg16, loss, optimizer)
    test(test_loader, vgg16, loss)
print("Done!")

Epoch 1 
------------------------
loss: 12.194332  [    0]/50000
loss: 1.277968  [ 3200]/50000
loss: 0.745037  [ 6400]/50000
loss: 0.423167  [ 9600]/50000
loss: 0.752071  [12800]/50000
loss: 0.537027  [16000]/50000
loss: 0.561714  [19200]/50000
loss: 0.680601  [22400]/50000
loss: 0.378994  [25600]/50000
loss: 0.865023  [28800]/50000
loss: 0.710256  [32000]/50000
loss: 0.401613  [35200]/50000
loss: 0.709614  [38400]/50000
loss: 0.631146  [41600]/50000
loss: 0.673939  [44800]/50000
loss: 0.638816  [48000]/50000
Test Error: 
 Accuracy: 82.0%, Avg loss: 0.515887

Epoch 2 
------------------------
loss: 0.423316  [    0]/50000
loss: 0.652893  [ 3200]/50000
loss: 0.767003  [ 6400]/50000
loss: 0.635063  [ 9600]/50000
loss: 0.393246  [12800]/50000
loss: 0.404183  [16000]/50000
loss: 0.974235  [19200]/50000
loss: 0.439563  [22400]/50000
loss: 0.376791  [25600]/50000
loss: 0.995656  [28800]/50000
loss: 0.744674  [32000]/50000
loss: 0.470045  [35200]/50000
loss: 0.901406  [38400]/50000
loss: 0.52

In [ ]:
model.eval()

with torch.no_grad():
  for test_img, test_label in test_loader:
    test_img, test_label = test_img.to(DEVICE), test_label.to(DEVICE)
    outputs = vgg16(test_img)
    break

    for i in range(3,4)
    output = outputs[i].unsqueeze(0)
    pred_proba = nn.Softmax(dim=1)(output)
    pred_class = pred_proba.argmax(dim=1).item()
    print(f'예측된 클래스: {pred_class} ({labels_map[pred_class]}')
    print(f'실제 클래스: {test_label[i].item()} ({labels_map[test_label[i].item()]}')

    plt.imshow(test_img[i].cpu())
    plt.title(f'predicted: {labels_map[pred_class]}')
    plt.show()